In [7]:
# CaisisImport. A utility to prepare a Caisis-to-Excel export for import into Oncoscape (via cBioPortal format).

# Steps:
# . ExportAsTSVs
# . ZeroDates
# . LimitByDiseaseGroups
# . 


In [9]:
import os
import errno
def save_file_and_folder(filename, obj):
    if not os.path.exists(os.path.dirname(filename)):
        try:
            os.makedirs(os.path.dirname(filename))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

    with open(filename, "w") as f:
        f.write(obj)

def patients_and_descriptive_header(df:pd.DataFrame, header):
    #ids = df['PatientId'].values
    #str_ids = "\n".join(ids)
    str_ids = df.to_string()
    a =  header + "\n" + str_ids
    return a

In [ ]:
from openpyxl import Workbook
from openpyxl import load_workbook
import pandas as pd
import os
import errno
import sys

dataset_folder = '../Caisis_NonPublicData/Brain'
diseaseChosen = 'Brain'

knownTables = ['Demographics', 'Status', 'Encounters', 'ClinicalStage', 'LabTests', 'PathTest', 'SocialHistory', 'LabTestGenetics', 'PathStageGrade', 'RadiationTherapy',  ]
#print(knownTables)
status_fullpath = dataset_folder +'/00_raw_csv/Status.csv'
if not os.path.exists(dataset_folder +'/00_raw_csv'):
    sys.exit('ERROR: Cannot find folder "00_raw_csv".')

if not os.path.exists(status_fullpath):
    sys.exit('ERROR: Status.csv file not found at ' + status_fullpath)

table_demographics = pd.read_csv(status_fullpath) #'Demographics.csv')
table_demographics['Date'] =pd.to_datetime(table_demographics.StatusDate)
table_demographics['PatientId'] =  table_demographics['PatientId'].astype(str)

dxRows = table_demographics[table_demographics.Status.eq('Diagnosis Date')]
diseaseRows = dxRows[dxRows.StatusDisease.eq(diseaseChosen)].sort_values('Date')

dxNanRows = diseaseRows[diseaseRows.Date.isna()][['PatientId']]
if(dxNanRows.size > 0):
    print('CHECK NoDxDate? REPORT NoDxDate.txt has *' +str(dxNanRows.size)+'* diagnoses of '+diseaseChosen+' without diagnosis dates.')
    report_body = patients_and_descriptive_header(dxNanRows, 'Patients with "'+diseaseChosen+" but no DiagnosisDate:")
#    save_file_and_folder('reports/NoDxDate.txt', report_body)
    save_file_and_folder(dataset_folder+'/reports/NoDxDate.txt', report_body)
else:
    print('CHECK NoDxDate? OK')
    print('TBD: delete existing NoDxDate.txt report.')
    if os.path.exists(dataset_folder+'/reports/NoDxDate.txt'):
        os.remove(dataset_folder+'/reports/NoDxDate.txt')
        print('REMOVED NoDxDate.txt')
        
diseaseDatedRows = diseaseRows[diseaseRows.Date.isna()==False]
#print(diseaseRows[diseaseRows.PatientId.eq('1920')])
print('----')
patientid_date_dict = {}
for index, row in diseaseDatedRows.iterrows():
    pid = str(row['PatientId'])
    if((pid in patientid_date_dict) == False):
        patientid_date_dict[pid] = row['Date']
    else:
        pass
print('Resulting patient IDs = ' + str(len(patientid_date_dict)))   
#patients_first_dx = pd.DataFrame.from_dict(patientid_date_dict, orient='index', columns=['PatientId'])
#patients_first_dx.set_index('PatientId', inplace=True)  
#patients_first_dx

data = []
for key in patientid_date_dict.keys():
    new_row = [key, patientid_date_dict[key]]
    data.append(new_row)
patients_first_dx = pd.DataFrame(data, columns=["PatientId", "DiagnosisDate"])
report_body = patients_and_descriptive_header(patients_first_dx, 'Patients First Diagnosis Date')

  
save_file_and_folder(dataset_folder+'/reports/PatientsFirstDx.txt', report_body)
patients_first_dx
